In [ ]:
# Accion Wells Fargo
tickerSymbol = "WFC"

# Rango de fecha
startDate = '2018-05-15' #  strings
endDate = '2023-05-15' #  strings

In [ ]:
# Librerias
import sys 
import yfinance as yf
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Neural Net modules
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
# Accion Wells Fargo
tickers = ["WFC", "^VIX", "^TNX", "GLD", "^BCOM", "^FVX", "DX-Y.NYB"]
data = yf.download(tickers, start="2018-05-15", end="2023-05-15", group_by="ticker")
print(data)

[*********************100%***********************]  7 of 7 completed
                 ^VIX                                                     \
                 Open       High        Low      Close  Adj Close Volume   
Date                                                                       
2018-05-15  13.130000  15.010000  12.500000  14.630000  14.630000    0.0   
2018-05-16  14.380000  14.910000  13.210000  13.420000  13.420000    0.0   
2018-05-17  13.540000  13.860000  12.650000  13.430000  13.430000    0.0   
2018-05-18  13.180000  13.870000  13.060000  13.420000  13.420000    0.0   
2018-05-21  12.990000  13.590000  12.780000  13.080000  13.080000    0.0   
...               ...        ...        ...        ...        ...    ...   
2023-05-08  17.730000  17.879999  16.830000  16.980000  16.980000    0.0   
2023-05-09  17.290001  17.860001  17.219999  17.709999  17.709999    0.0   
2023-05-10  17.580000  18.309999  16.360001  16.940001  16.940001    0.0   
2023-05-11  16.7999

In [ ]:
type(data)

pandas.core.frame.DataFrame

In [ ]:
data.head()

^VIX                                         DX-Y.NYB             \
             Open   High    Low  Close Adj Close Volume       Open       High   
Date                                                                            
2018-05-15  13.13  15.01  12.50  14.63     14.63    0.0  92.650002  93.459999   
2018-05-16  14.38  14.91  13.21  13.42     13.42    0.0  93.309998  93.629997   
2018-05-17  13.54  13.86  12.65  13.43     13.43    0.0  93.260002  93.570000   
2018-05-18  13.18  13.87  13.06  13.42     13.42    0.0  93.500000  93.830002   
2018-05-21  12.99  13.59  12.78  13.08     13.08    0.0  93.669998  94.059998   

                                  ...      ^BCOM                               \
                  Low      Close  ...        Low      Close  Adj Close Volume   
Date                              ...                                           
2018-05-15  92.610001  93.220001  ...  89.629997  90.120003  90.120003    0.0   
2018-05-16  93.139999  93.389999  ...  89.809998  90.180000  90.180000    0.0   
2018-05-17  93.120003  93.470001  ...  90.230003  90.309998  90.309998    0.0   
2018-05-18  93.339996  93.639999  ...  90.349998  90.410004  90.410004    0.0   
2018-05-21  93.510002  93.680000  ...  90.510002  90.970001  90.970001    0.0   

                  WFC                                                          
                 Open       High        Low      Close  Adj Close      Volume  
Date                                                                           
2018-05-15  54.430000  55.279999  54.380001  54.750000  47.358711  14991100.0  
2018-05-16  54.700001  55.320000  54.560001  55.040001  47.609566  16450200.0  
2018-05-17  54.400002  54.669998  54.029999  54.220001  46.900257  21622400.0  
2018-05-18  54.150002  54.419998  53.689999  53.700001  46.450451  21492700.0  
2018-05-21  54.080002  54.630001  54.070000  54.540001  47.177055  11491400.0  

[5 rows x 42 columns]

In [ ]:
#Merging multi index columns
data.columns = data.columns.map('|'.join).str.strip('|')

print(data)

            ^VIX|Open  ^VIX|High   ^VIX|Low  ^VIX|Close  ^VIX|Adj Close  \
Date                                                                      
2018-05-15  13.130000  15.010000  12.500000   14.630000       14.630000   
2018-05-16  14.380000  14.910000  13.210000   13.420000       13.420000   
2018-05-17  13.540000  13.860000  12.650000   13.430000       13.430000   
2018-05-18  13.180000  13.870000  13.060000   13.420000       13.420000   
2018-05-21  12.990000  13.590000  12.780000   13.080000       13.080000   
...               ...        ...        ...         ...             ...   
2023-05-08  17.730000  17.879999  16.830000   16.980000       16.980000   
2023-05-09  17.290001  17.860001  17.219999   17.709999       17.709999   
2023-05-10  17.580000  18.309999  16.360001   16.940001       16.940001   
2023-05-11  16.799999  18.190001  16.629999   16.930000       16.930000   
2023-05-12  16.830000  17.920000  16.379999   17.030001       17.030001   

            ^VIX|Volume 

In [ ]:
data.head()

,^VIX|Open,^VIX|High,^VIX|Low,^VIX|Close,^VIX|Adj Close,^VIX|Volume,DX-Y.NYB|Open,DX-Y.NYB|High,DX-Y.NYB|Low,DX-Y.NYB|Close,...,^BCOM|Low,^BCOM|Close,^BCOM|Adj Close,^BCOM|Volume,WFC|Open,WFC|High,WFC|Low,WFC|Close,WFC|Adj Close,WFC|Volume
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-15,13.13,15.01,12.50,14.63,14.63,0.0,92.650002,93.459999,92.610001,93.220001,...,89.629997,90.120003,90.120003,0.0,54.430000,55.279999,54.380001,54.750000,47.358711,14991100.0
2018-05-16,14.38,14.91,13.21,13.42,13.42,0.0,93.309998,93.629997,93.139999,93.389999,...,89.809998,90.180000,90.180000,0.0,54.700001,55.320000,54.560001,55.040001,47.609566,16450200.0
2018-05-17,13.54,13.86,12.65,13.43,13.43,0.0,93.260002,93.570000,93.120003,93.470001,...,90.230003,90.309998,90.309998,0.0,54.400002,54.669998,54.029999,54.220001,46.900257,21622400.0
2018-05-18,13.18,13.87,13.06,13.42,13.42,0.0,93.500000,93.830002,93.339996,93.639999,...,90.349998,90.410004,90.410004,0.0,54.150002,54.419998,53.689999,53.700001,46.450451,21492700.0
2018-05-21,12.99,13.59,12.78,13.08,13.08,0.0,93.669998,94.059998,93.510002,93.680000,...,90.510002,90.970001,90.970001,0.0,54.080002,54.630001,54.070000,54.540001,47.177055,11491400.0


In [ ]:
# Checking null value
data.isnull().sum()

^VIX|Open             4
^VIX|High             4
^VIX|Low              4
^VIX|Close            4
^VIX|Adj Close        4
^VIX|Volume           4
DX-Y.NYB|Open         0
DX-Y.NYB|High         0
DX-Y.NYB|Low          0
DX-Y.NYB|Close        0
DX-Y.NYB|Adj Close    0
DX-Y.NYB|Volume       0
GLD|Open              4
GLD|High              4
GLD|Low               4
GLD|Close             4
GLD|Adj Close         4
GLD|Volume            4
^TNX|Open             4
^TNX|High             4
^TNX|Low              4
^TNX|Close            4
^TNX|Adj Close        4
^TNX|Volume           4
^FVX|Open             4
^FVX|High             4
^FVX|Low              4
^FVX|Close            4
^FVX|Adj Close        4
^FVX|Volume           4
^BCOM|Open            4
^BCOM|High            4
^BCOM|Low             4
^BCOM|Close           4
^BCOM|Adj Close       4
^BCOM|Volume          4
WFC|Open              4
WFC|High              4
WFC|Low               4
WFC|Close             4
WFC|Adj Close         4
WFC|Volume      

In [ ]:
# Checking na value
data.isna().any()
#To do - eliminate null values, replace them, check

^VIX|Open              True
^VIX|High              True
^VIX|Low               True
^VIX|Close             True
^VIX|Adj Close         True
^VIX|Volume            True
DX-Y.NYB|Open         False
DX-Y.NYB|High         False
DX-Y.NYB|Low          False
DX-Y.NYB|Close        False
DX-Y.NYB|Adj Close    False
DX-Y.NYB|Volume       False
GLD|Open               True
GLD|High               True
GLD|Low                True
GLD|Close              True
GLD|Adj Close          True
GLD|Volume             True
^TNX|Open              True
^TNX|High              True
^TNX|Low               True
^TNX|Close             True
^TNX|Adj Close         True
^TNX|Volume            True
^FVX|Open              True
^FVX|High              True
^FVX|Low               True
^FVX|Close             True
^FVX|Adj Close         True
^FVX|Volume            True
^BCOM|Open             True
^BCOM|High             True
^BCOM|Low              True
^BCOM|Close            True
^BCOM|Adj Close        True
^BCOM|Volume        

In [ ]:
data.dropna(inplace=True)
data.isna().any()

^VIX|Open             False
^VIX|High             False
^VIX|Low              False
^VIX|Close            False
^VIX|Adj Close        False
^VIX|Volume           False
DX-Y.NYB|Open         False
DX-Y.NYB|High         False
DX-Y.NYB|Low          False
DX-Y.NYB|Close        False
DX-Y.NYB|Adj Close    False
DX-Y.NYB|Volume       False
GLD|Open              False
GLD|High              False
GLD|Low               False
GLD|Close             False
GLD|Adj Close         False
GLD|Volume            False
^TNX|Open             False
^TNX|High             False
^TNX|Low              False
^TNX|Close            False
^TNX|Adj Close        False
^TNX|Volume           False
^FVX|Open             False
^FVX|High             False
^FVX|Low              False
^FVX|Close            False
^FVX|Adj Close        False
^FVX|Volume           False
^BCOM|Open            False
^BCOM|High            False
^BCOM|Low             False
^BCOM|Close           False
^BCOM|Adj Close       False
^BCOM|Volume        

In [ ]:
data.columns

Index(['^VIX|Open', '^VIX|High', '^VIX|Low', '^VIX|Close', '^VIX|Adj Close',
       '^VIX|Volume', 'DX-Y.NYB|Open', 'DX-Y.NYB|High', 'DX-Y.NYB|Low',
       'DX-Y.NYB|Close', 'DX-Y.NYB|Adj Close', 'DX-Y.NYB|Volume', 'GLD|Open',
       'GLD|High', 'GLD|Low', 'GLD|Close', 'GLD|Adj Close', 'GLD|Volume',
       '^TNX|Open', '^TNX|High', '^TNX|Low', '^TNX|Close', '^TNX|Adj Close',
       '^TNX|Volume', '^FVX|Open', '^FVX|High', '^FVX|Low', '^FVX|Close',
       '^FVX|Adj Close', '^FVX|Volume', '^BCOM|Open', '^BCOM|High',
       '^BCOM|Low', '^BCOM|Close', '^BCOM|Adj Close', '^BCOM|Volume',
       'WFC|Open', 'WFC|High', 'WFC|Low', 'WFC|Close', 'WFC|Adj Close',
       'WFC|Volume'],
      dtype='object')

In [ ]:
#Checking type index
type(data.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [ ]:
data['Date'] = pd.to_datetime(data.index.values, format = '%m/%d/%Y')
data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data.shape

(1258, 42)

In [ ]:
data['WFC|Close'] = data['WFC|Close'].astype(float)
data['WFC|Close'] = pd.to_numeric(data['WFC|Close'])
data['Date'] = pd.to_datetime(data['Date'])

# Convertir la columna 'Fecha' al tipo datetime
data['Date'] = pd.to_datetime(data['Date'])

# Establecer la columna 'Fecha' como índice
data.set_index ('Date', inplace=True)


KeyError: ignored

In [ ]:
#Let's select our features
x = data[['^VIX|Close','GLD|Close','WFC|Open','WFC|High','WFC|Low','WFC|Adj Close','WFC|Volume','^BCOM|Close','DX-Y.NYB|Close','^FVX|Close','^TNX|Close']]
y = data['WFC|Close']

In [ ]:
def SplitData(data, train_pct, shift):
  # inputs: dataframe , training_pct (float between 0 and 1), size of the lag (int)
  # ouptut: x train dataframe, y train data frame, x test dataframe, y test dataframe, train data frame, test dataframe

  train_pct = .75
  train_pt = int(len(data)*train_pct)
  
  
  train = data.iloc[:train_pt,:]
  test = data.iloc[train_pt:,:]
  
  x_train = train.iloc[:shift,1:-1]
  y_train = train['WFC|Close_lag'][:shift]
  x_test = test.iloc[:shift,1:-1]
  y_test = test['WFC|Close'][:shift]


  return x_train, y_train, x_test, y_test, train, test

In [ ]:
 train_pct = .75

In [ ]:
# Nueva columna al DataFrame "Close_lag" que contiene los valores de la columna "Close" desplazados hacia atrás en shiftdays días. Esto puede ser útil para realizar análisis de series de tiempo, cálculos de diferencias entre valores consecutivos o construir modelos predictivos que utilicen valores pasados como características
# Crear los "lags ""
def CreateLags(data,lag_size):
  # inputs: dataframe , size of the lag (int)
  # ouptut: dataframe ( with extra lag column), shift size (int)

  # add lag
  shiftdays = lag_size
  shift = -shiftdays
  data['WFC|Close_lag'] = data['WFC|Close'].shift(shift)
  return data, shift

In [ ]:
# Modelo Base regresion lineal 

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def LinearRegression_fnc(x_train, y_train, x_test, y_test, cv=5):
    # Inputs: x train data, y train data, x test data, y test data (all dataframes)
    #        cv: número de folds en la validación cruzada (por defecto, 5)
    # Output: the predicted values for the test data (list)

    lr = LinearRegression()
    lr.fit(x_train, y_train)

    # Realizar la validación cruzada y obtener el rendimiento del modelo
    scores = cross_val_score(lr, x_train, y_train, cv=cv, scoring='r2')
    lr_R2_cv = scores.mean()
    lr_MSE_cv = -scores.mean()

    lr_pred = lr.predict(x_test)
    lr_MSE = mean_squared_error(y_test, lr_pred)
    lr_R2 = lr.score(x_test, y_test)
    print('Linear Regression R2: {}'.format(lr_R2))
    print('Linear Regression MSE: {}'.format(lr_MSE))
    print('Linear Regression R2 (CV): {}'.format(lr_R2_cv))
    print('Coeficiente: ', lr.coef_)
    print('Intercepto: ', lr.intercept_)

    return lr_pred

In [ ]:
# Funcion para red neuronal

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

def ANN_func(x_train, y_train, x_test, y_test):
    # Scaling data
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    MLP = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(100,),
                       activation='identity', learning_rate='adaptive')

    MLP.fit(x_train_scaled, y_train)

    # Perform cross-validation
    MLP_cv_scores = cross_val_score(MLP, x_train_scaled, y_train, cv=5)
    MLP_cv_R2 = MLP_cv_scores.mean()
    MLP_cv_MSE = -MLP_cv_scores.mean()

    MLP_pred = MLP.predict(x_test_scaled)
    MLP_MSE = mean_squared_error(y_test, MLP_pred)
    MLP_R2 = MLP.score(x_test_scaled, y_test)

    print('Multi-layer Perceptron R2 Test: {}'.format(MLP_R2))
    print('Multi-layer Perceptron MSE: {}'.format(MLP_MSE))
    print('Multi-layer Perceptron R2 (CV): {}'.format(MLP_cv_R2))

    return MLP_pred


In [ ]:
# Parametrizacion de dias (Shifts)
shifts = [1,5,10,30]

In [ ]:

!{sys.executable} -m pip install sklearn # Dsklearn 
from sklearn.metrics import mean_squared_error # error cuadratico medio
from sklearn.linear_model import LinearRegression # Modelo regresion lineal
from sklearn.neural_network import MLPRegressor # IModelo red neuronal artificial 
from sklearn.preprocessing import StandardScaler # Funcion para escalar y desescalar la red neuronal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
data.dropna(inplace=True) #Depurando Nulos

In [ ]:
# Parametros del PLOT
w = 16 # width
h = 4 # height 

# Function para hacer los plots
def PlotModelResults_Plotly(train, test, pred, ticker, w, h, shift_days,name):
  # inputs: train dataframe, test dataframe, predicted value (list), ticker ('string'), width (int), height (int), shift size (int), name (string)
  # output: None

  # Create lines of the training actual, testing actual, prediction 
  D1 = go.Scatter(x=train.index,y=train['WFC|Close'],name = 'Train Actual') # Training actuals
  D2 = go.Scatter(x=test.index[:shift],y=test['WFC|Close'],name = 'Test Actual') # Testing actuals
  D3 = go.Scatter(x=test.index[:shift],y=pred,name = 'Our Prediction') # Testing predction

  # Combine in an object  
  line = {'data': [D1,D2,D3],
          'layout': {
              'xaxis' :{'title': 'Date'},
              'yaxis' :{'title': '$'},
              'title' : name + ' - ' + tickerSymbol + ' - ' + str(shift_days)
          }}
  # Send object to a figure 
  fig = go.Figure(line)

  # Show figure
  fig.show()

In [ ]:
import numba
import numpy as np
import pandas as pd
import datetime

In [ ]:
## Modelando con cada periodo. Para cada accion, tendremos una predicción para cada duración de tiempo en el futuro


for j in shifts: 
  print(str(j) + ' days out:')
  print('------------')
  data_lag, shift = CreateLags(data,j)
  #data_lag = CorrectColumnTypes(data_lag)
  x_train, y_train, x_test, y_test, train, test = SplitData(data, train_pct, shift)

  # Linear Regression
  print("Linear Regression")
  lr_pred = LinearRegression_fnc(x_train,y_train, x_test, y_test)
  PlotModelResults_Plotly(train, test, lr_pred, tickerSymbol, w, h, j, 'Linear Regression')

  # Artificial Neuarl Network 
  print("ANN")
  MLP_pred = ANN_func(x_train,y_train, x_test, y_test)
  PlotModelResults_Plotly(train, test, MLP_pred, tickerSymbol, w, h, j, 'ANN')
  print('------------')

1 days out:
------------
Linear Regression
Linear Regression R2: 0.9553630431572131
Linear Regression MSE: 0.8116652649781644
Linear Regression R2 (CV): 0.9539954012939544
Coeficiente:  [ 5.46219480e-03 -5.45735568e-02  1.92544371e-02  1.92521054e-02
 -4.20554071e-10 -1.78517739e-02 -2.12881297e-02  9.30355856e-02
 -7.63981485e-03 -7.63981487e-03  3.25624597e-12  1.05135665e-02
 -8.85617470e-02  2.59475923e-02  2.28249383e-02  2.28249383e-02
  5.98711897e-10 -8.87415085e+00  1.13355250e+01  3.74429707e+00
 -2.99951982e+00 -2.99951982e+00 -5.14588372e-13  7.75516675e+00
 -7.19490849e+00 -3.26969503e+00  1.20789213e+00  1.20789213e+00
  0.00000000e+00  1.11753242e-01 -1.22176941e-01 -2.00076867e-01
  1.20393160e-01  1.20393160e-01  0.00000000e+00 -2.70140205e-01
  1.50072260e-01  4.28494747e-01  7.09175212e-01 -5.66647587e-02
 -3.05637127e-10]
Intercepto:  -3.382767507982244


ANN
Multi-layer Perceptron R2 Test: 0.9745878113136811
Multi-layer Perceptron MSE: 0.4620877479708657
Multi-layer Perceptron R2 (CV): 0.9278732334235468


------------
5 days out:
------------
Linear Regression
Linear Regression R2: 0.3055031668099535
Linear Regression MSE: 12.479511512877771
Linear Regression R2 (CV): 0.7509398630344497
Coeficiente:  [ 2.97774584e-02 -4.36844397e-03 -2.76802943e-02 -2.76802881e-02
 -1.39753487e-12 -4.82374161e-01  1.84558555e-01 -2.80335076e-01
  3.69446700e-01  3.69446700e-01  3.43058915e-14 -2.24975069e-01
 -3.86810512e-02  4.67909353e-01 -1.17598138e-01 -1.17598138e-01
  3.95120685e-09  2.34642770e+00 -3.81458163e+00 -4.58496871e+00
  3.48790678e+00  3.48790678e+00  4.44089210e-16 -1.52040467e+01
  1.70511194e+01  1.88657688e+01 -1.11227148e+01 -1.11227148e+01
 -1.77635684e-15 -5.09483695e-01  7.31333601e-01  3.72639267e-01
 -2.31795600e-01 -2.31795600e-01  0.00000000e+00 -6.08550334e-01
  3.67547882e-01  8.43431969e-01  4.92412536e-01 -2.65653822e-01
 -1.00296516e-08]
Intercepto:  -13.244947463963157


ANN
Multi-layer Perceptron R2 Test: 0.26744191061866185
Multi-layer Perceptron MSE: 13.163439591645318
Multi-layer Perceptron R2 (CV): 0.7254960244352109


------------
10 days out:
------------
Linear Regression
Linear Regression R2: -0.6972732937826545
Linear Regression MSE: 30.003261221449918
Linear Regression R2 (CV): 0.4986041601574646
Coeficiente:  [-2.48550289e-02 -4.27883781e-02  1.75436663e-03  1.75389824e-03
  3.49056201e-12 -9.88089900e-01  1.22363964e+00  2.09777288e-01
 -1.00980504e-01 -1.00980504e-01  4.26325641e-14 -2.37761935e-01
  1.47148897e-01  3.33147544e-01 -1.56336625e-01 -1.56336625e-01
 -5.48452574e-08  1.29521640e+01 -7.96944642e+00 -2.01120436e+01
  8.19821005e+00  8.19821005e+00 -1.77635684e-15 -2.26624853e+01
  1.60638509e+01  2.94690250e+01 -1.28307456e+01 -1.28307456e+01
  3.55271368e-15 -2.53114488e-01  3.59421305e-01  2.08884300e-01
 -2.97140539e-02 -2.97140539e-02  0.00000000e+00 -7.47419031e-01
  8.08745515e-01  4.76011212e-01  7.30733227e-01 -6.04494243e-01
 -2.03398548e-08]
Intercepto:  -17.421706490505308


ANN
Multi-layer Perceptron R2 Test: -0.22789166783196801
Multi-layer Perceptron MSE: 21.705847017423242
Multi-layer Perceptron R2 (CV): 0.5043286805791095


------------
30 days out:
------------
Linear Regression
Linear Regression R2: -2.8600208160582126
Linear Regression MSE: 65.62371873938123
Linear Regression R2 (CV): -0.6615856274340637
Coeficiente:  [ 5.02143822e-02 -3.19631679e-02 -5.42020049e-02 -5.42020664e-02
 -5.08431838e-12  1.38691177e-01 -8.93488898e-01  2.58754024e-01
  2.87898949e-01  2.87898949e-01 -1.39888101e-13 -3.55790104e-01
  2.41052696e-01  3.82734645e-01 -2.49870140e-01 -2.49870140e-01
 -6.14479383e-08  7.96932764e+00 -3.01677348e+01 -8.40124322e+00
  1.58405472e+01  1.58405472e+01  1.03139719e-13 -3.47730723e+01
  6.63963539e+01  1.27015589e+01 -2.57658344e+01 -2.57658344e+01
  2.13162821e-14  3.21747265e-01  2.67168668e-01 -3.38680723e-01
  1.89055158e-01  1.89055158e-01  0.00000000e+00 -4.49573905e-01
  3.28159962e-01  5.67018759e-01  1.75861469e+00 -1.99150688e+00
 -1.40621697e-08]
Intercepto:  13.095554869463196


ANN
Multi-layer Perceptron R2 Test: -1.2701077197441046
Multi-layer Perceptron MSE: 38.59381013927117
Multi-layer Perceptron R2 (CV): -0.8140324487662806


------------


In [ ]:
## Mirando cada variable con su coeficiente
from sklearn.linear_model import LinearRegression

# Crear el objeto de regresión lineal
lr = LinearRegression()

# Ajustar el modelo utilizando los datos de entrenamiento
lr.fit(x_train, y_train)

# Obtener los nombres de las variables independientes (columnas) del DataFrame
column_names = x_train.columns.tolist()

# Obtener los coeficientes del modelo
coefficients = lr.coef_

# Imprimir el nombre de la variable y el coeficiente correspondiente
for variable, coef in zip(column_names, coefficients):
    print("Variable:", variable)
    print("Coeficiente:", coef)
    print("---")

In [ ]:
import yfinance as yf
import pandas as pd



# Crear un diccionario para almacenar las matrices de correlación
correlation_matrices = {}

# Calcular la matriz de correlación
correlation_matrix = data.corr()

    # Almacenar la matriz de correlación en el diccionario
correlation_matrices[tickerSymbol] = correlation_matrix

# Imprimir las matrices de correlación
for tickerSymbol, correlation_matrix in correlation_matrices.items():
    print("Ticker:", tickerSymbol)
    print(correlation_matrix)
    print("---------------------")

In [ ]:
import matplotlib.pyplot as plt

# Iterar sobre las matrices de correlación
for tickerSymbol, correlation_matrix in correlation_matrices.items():
    # Crear una figura y un eje
    fig, ax = plt.subplots()
    
    # Crear un mapa de calor de la matriz de correlación
    cax = ax.matshow(correlation_matrix, cmap='coolwarm')
    
    # Agregar una barra de color
    fig.colorbar(cax)
    
    # Establecer las etiquetas de los ejes x e y
    ax.set_xticklabels([''] + correlation_matrix.columns, rotation=90)
    ax.set_yticklabels([''] + correlation_matrix.columns)
    
    # Mostrar el título del gráfico
    ax.set_title('Correlation Matrix: {}'.format(tickerSymbol))
    
    # Mostrar el gráfico
    plt.show()